In [ ]:
import tensorflow as tf

## Keras高层接口

Keras 是一个主要由 Python 语言开发的开源神经网络计算库，最初由 François Chollet编写， 它被设计为高度模块化和易扩展的高层神经网络接口，使得用户可以不需要过多的专业知识就可以简洁、 快速地完成模型的搭建与训练。 Keras 库分为前端和后端，其中后端可以基于现有的深度学习框架实现，如 Theano， CNTK， TensorFlow，前端接口即 Keras抽象过的统一接口 API。用户通过 Keras 编写的代码可以轻松的切换不同的后端运行，灵活性较大。 

那么 Keras 与 tf.keras 有什么区别与联系呢？其实 Keras 可以理解为一套搭建与训练神经网络的高层 API 协议， Keras 本身已经实现了此协议，可以方便的调用 TensorFlow，CNTK 等后端完成加速计算；在 TensorFlow 中，也实现了一套 Keras 协议， 即 tf.keras，但只能基于 TensorFlow 后端计算， 并对 TensorFlow 的支持更好。 对于使用 TensorFlow 的开发者来说， tf.keras 可以理解为一个普通的子模块，与其他子模块，如 tf.math， tf.data 等并没有什么差别。

### 1.常见功能模块
- 常见数据集加载函数
- 网络层类
- 模型容器
- 损失函数类
- 优化器类
- 经典模型类
- ……

数据集包括：
- Boston房价预测数据集
- CIFAR图片数据集
- MNIST/FashionMNIST手写数字图片数据集 
- IMDB文本数据集等。

#### 1.1 常见网络层类
通常在tf.keras.layers命名空间下提供了大量常见网络层的类接口
- 全连接层
- 激活含水层
- 池化层
- 卷积层
- 循环神经网络层
- ……

In [ ]:
# softmax层
x = tf.constant([0.,3,2,1,5,3])
softmax_layer = tf.keras.layers.Softmax(axis = -1)
softmax_layer(x)

#### 1.2 网络容器
对于常见的网络，需要手动调用每一层的类实例完成前向传播运算，当网络层数变得较深时， 这一部分代码显得非常臃肿。可以通过 Keras 提供的网络容器 Sequential 将多个网络层封装成一个大网络模型，只需要调用网络模型的实例一次即可完成数据从第一层到最末层的顺序运算。

In [ ]:
# 通过Sequential进行封装
network = tf.keras.Sequential([
    tf.keras.layers.Dense(3),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(2),
    tf.keras.layers.ReLU()
])
x = tf.random.normal([4, 3])
network(x)

In [ ]:
# Sequential容器可以通过add()方法继续追加新的网络层
network = tf.keras.Sequential([])
network.add(tf.keras.layers.Dense(3))
network.add(tf.keras.layers.ReLU())

# 通过build方法可以设置输入的大小
network.build(input_shape = (None, 5))
network.summary()

In [ ]:
# 查看所有层的待优化张量
for p in network.trainable_variables:
    print(p.name, p.shape)

### 2. 模型装配、训练与测试
在训练网络时，一般的流程是通过前向计算获得网络的输出值， 再通过损失函数计算网络误差，然后通过自动求导工具计算梯度并更新，同时间隔性地测试网络的性能。
#### 2.1 模型装配


In [ ]:
# 首先创建网络
from tensorflow.keras import layers,Sequential
network = Sequential([
    layers.Dense(256, activation = 'relu'),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])
network.build(input_shape = (None, 784))
for p in network.trainable_variables:
    print(p.name, p.shape)

创建网络后，正常的流程是通过循环迭代数据集多遍，每次按批产生训练数据， 前向计算，然后通过损失函数计算误差值，并反向传播自动计算梯度，更新网络参数。这一部分逻辑由于非常通用，在 keras 中提供了 compile()和 fit()函数方便实现上述逻辑。

In [ ]:
# 通过compile函数指定网络使用的优化器对象，损失函数，评价指标
from tensorflow.keras import optimizers, losses
network.compile(optimizer = optimizers.Adam(lr = 0.01), # 优化器
               loss = losses.CategoricalCrossentropy(from_logits = True), # 损失函数
               metrics = ['acc'] # 评价指标
               )

#### 2.2 模型训练
模型装配完成后，即可通过 fit()函数送入待训练的数据和验证用的数据集

In [ ]:
# fit
history = network.fit(train_db, # tf.data.Dataset对象，也可以是Numpy Array类型
                      epochs = 10,  # 迭代次数
                      validation_data = val_db, # 指定用于验证的数据集
                      validation_freq = 2 # 每几次迭代进行一次验证
                     )
# history.history为字典对象，包含了训练过程中的记录
history.history

#### 2.3 模型测试
Model 基类除了可以便捷地完成网络的装配与训练、验证，还可以非常方便的预测和测试。

In [ ]:
# 加载一个batch的测试数据
x, y = next(iter(db_test))
out = network.predict(x) # 预测模型
network.evaluate(db_test) # 模型测试

### 3. 模型保存与加载
模型训练完成后，需要将模型保存到文件系统上，从而方便后续的模型测试与部署工作。实际上， 在训练时间隔性地保存模型状态也是非常好的习惯，这一点对于训练大规模的网络尤其重要，一般大规模的网络需要训练数天乃至数周的时长，一旦训练过程被中断或者发生宕机等意外，之前训练的进度将全部丢失。如果能够间断的保存模型状态到文系统，即使发生宕机等意外，也可以从最近一次的网络状态文件中恢复，从而避免浪费大量的训练时间。 因此模型的保存与加载非常重要。

#### 3.1 张量方式
save_weights(path)方法保存

In [ ]:
# save_weights
network.save_weights('weights.ckpt') # 将network模型保存到weights.ckpt文件上

# 删除网络对象
del network

In [ ]:
# 重新创建相同的网络结构
network = Sequential([
    layers.Dense(256, activation = 'relu'),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])
network.compile(optimizer = optimizers.Adam(lr = 0.01), # 优化器
               loss = losses.CategoricalCrossentropy(from_logits = True), # 损失函数
               metrics = ['acc'] # 评价指标
               )

# 从参数文件中读取数据并写入当前网络
network.load_weights('weights.ckpt')

文件中保存的仅仅是参数张量的数值，并没有其他额外的结构参数。但是它需要使用相同的网络结构才能够恢复网络状态，因此一般在拥有网络源文件的情况下使用。

#### 3.2 网络方式
通过 Model.save(path)函数可以将模型的结构以及模型的参数保存到一个 path 文件上，在不需要网络源文件的条件下，通过 keras.models.load_model(path)即可恢复网络结构和网络参数。

In [ ]:
# 保存模型结构和参数到文件
network.save('model.h5')
del network

In [ ]:
# 从文件恢复网络结构与网络参数
network = tf.keras.models.load_model('model.h5')

#### 3.3 SavedModel方式
当需要将模型部署到其他平台时，采用TensorFlow 提出的 SavedModel 方式更具有平台无关性。

In [ ]:
# 保存模型结构与模型参数到文件
tf.keras.experimental.export_saved_model(network, 'model_savedmodel')
del network

In [ ]:
# 从文件恢复网络结构与网络参数
network = tf.keras.experimental.load_from_saved_model('model_savedmodel')